In [1]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

In [2]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

In [3]:
def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

In [4]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (64, 64), cv2.INTER_LINEAR) #INTER_LINEAR is algorithm to downsize image
    #I could try using INTER_AREA as, according to the URL below, is more beneficial in downsizing
    #http://tanbakuchi.com/posts/comparison-of-openv-interpolation-algorithms/
    return [path, resized]

In [5]:
def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

In [6]:
train = glob.glob("/Users/keerat/dev/AOSResearch/resources/Train/**/*.jpg")
train = pd.DataFrame([[p.split('/')[7],p.split('/')[8],p] for p in train], columns = ['type','image','path'])[::5] 
train = im_stats(train)
train = train[train['size'] != '0 0'].reset_index(drop=True) #corrupt images removed
print("loading train data")
train_data = normalize_image_features(train['path'])
print("train data loaded")
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
print(le.classes_) 
np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)

test = glob.glob("/Users/keerat/dev/AOSResearch/resources/test/*.jpg")
test = pd.DataFrame([[p.split('/')[7],p] for p in test], columns = ['image','path']) [::20] 
test.head(5)
print("loading test data")
test_data = normalize_image_features(test['path'])
np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)
print("test data loaded")
test_id = test.image.values
np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)

Bad images removed
loading train data
train data loaded
['Type_1' 'Type_2' 'Type_3']
loading test data
test data loaded


In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
K.set_image_dim_ordering('th')
K.set_floatx('float32')
np.random.seed(17)

/Users/keerat/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

In [14]:
def create_model(opt_='adamax'):
    model = Sequential()
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th', input_shape=(3, 64, 64))) 
    #Could try different input shape
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Dropout(0.2)) #Can change dropout rate
    #Going to add more layers before Wednesday
    model.add(Flatten())
    model.add(Dense(12, activation='tanh')) #Can try different function
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='softmax')) #Can try different function

    model.compile(optimizer=opt_, loss='sparse_categorical_crossentropy', metrics=['accuracy']) 
    return model

In [15]:
def cleanImages():
    datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
    datagen.fit(train_data)
    return datagen

In [21]:
def fitAndPredict():
    print("cleaning images")
    datagen=cleanImages()
    print("images cleaned")
    
    model = create_model()
    x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)
    print("fitting data")
    model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), nb_epoch=200, samples_per_epoch=len(x_train), verbose=20, validation_data=(x_val_train, y_val_train))
    print("data fitted in model")
    loss, accuracy = model.evaluate(train_data, train_target)
    print('loss: ', loss, '\naccuracy: ', accuracy)
    test_data = np.load('test.npy')
    test_id = np.load('test_id.npy')
    print("creating predictions")
    predictions = model.predict_proba(test_data)
    print("predictions made")
    return predictions

In [22]:
def test(isTrue):
    pred=fitAndPredict()
    print("creating test file")
    df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])
    df['image_name'] = test_id
    if (isTrue):
        df.to_csv('test.csv', index=False)
        print("Test file created in users/keerat/...")
    else:
        print(df.to_string())

In [23]:
if __name__ == '__main__':
    
    test(isTrue = False)

cleaning images
images cleaned


/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", input_shape=(3, 64, 64..., data_format="channels_first")`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  after removing the cwd from sys.path.
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", data_format="channels_first")`
  """
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first"

fitting data


/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
/Users/keerat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=20, validation_data=(array([[[..., steps_per_epoch=11, epochs=200)`
  if __name__ == '__main__':


Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71/200
Epoch 72/200
Epoch 73/200
Epoch 74/200
Epoch 75/200
Epoch 76/200
Epoch 77/200
Epoch 78